Write an SQL script to create a data table to store the visitor assignment data

Write a python script to extract all the visitor assignment log messages from the log file and store it in the data table you created
Write the SQL queries that will help answer the questions A and B below

  A. What are the total number users assigned to the “Test” and “Control” groups in each experiment?

  B. Which day had the highest number of user group assignments per experiment?

In [2]:
import json,requests
from urllib2 import Request, urlopen
import pandas as pd
from pyspark.sql.types import StringType, StructField, StructType, BooleanType,DoubleType, ArrayType, IntegerType,TimestampType


#file_location = "/dbfs/FileStore/tables/lighthouse_logs-08bd3.log"
#with open(file_location,'rb') as F:
#  fcontent=F.readlines()


schema = StructType([
        StructField("level", StringType(), True),
        StructField("msg", StringType(), False),
        StructField("path", StringType(), False),
        StructField("process_time_ms", StringType(), False),
        StructField("proxy_reason", StringType(), False),
        StructField("request_params", StringType(), False),
        StructField("requestbackend", StringType(), False),
        StructField("time", StringType(), True),
       ])


fcontent=''
jsoncontent=[]

file_location = 'https://s3-ap-southeast-1.amazonaws.com/ms-data-coding-challenge/lighthouse-logs.log'
headers = {"Authorization": "Negotiate "}
res = requests.get(file_location, headers='', verify=False)
data=res.text

for line in data.split("\n"):
  if "invalid byte" not in line and 'error' not in line and line != '':
    jsoncontent.append(json.loads(line))


pdf = pd.DataFrame(jsoncontent)
df = spark.createDataFrame(pdf,schema=schema)
temp_table_name = "lighthouse_logs"
df.createOrReplaceTempView(temp_table_name)

/databricks/python/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
 InsecureRequestWarning)

In [3]:
%sql
-- Completed
SELECT 
EXPERIMENT_GRP,
COUNT(1) AS TOTAL_USERS
FROM (SELECT 
(CASE WHEN proxy_reason LIKE '%test%' OR msg LIKE '%Test%' THEN 'Test'
WHEN proxy_reason LIKE '%control%' OR  msg LIKE '%Control%' THEN 'Control'
END) AS EXPERIMENT_GRP,* FROM lighthouse_logs 
WHERE proxy_reason LIKE '%BucketID%' OR msg LIKE '%assigned%')
GROUP BY EXPERIMENT_GRP HAVING EXPERIMENT_GRP != 'null'  ORDER BY TOTAL_USERS;

EXPERIMENT_GRP,TOTAL_USERS
Control,5574
Test,5950


In [4]:
%sql
-- Completed
SELECT 
TO_DATE(CAST(UNIX_TIMESTAMP(substr(TIME,1, 10), 'yyyy-MM-dd') AS TIMESTAMP)) as DATE,
COUNT(1) AS TOTAL_USERS
FROM lighthouse_logs 
WHERE proxy_reason LIKE '%BucketID%' OR msg LIKE '%assigned%'
GROUP BY DATE 
ORDER BY TOTAL_USERS DESC limit 1;

DATE,TOTAL_USERS
2018-03-15,12107
